# Desktop DNS Notebook 3



## Check resolution and mesh quality

In [1]:
# reload modules
%reload_ext autoreload
%autoreload 2


## 3.1 Interpolating onto a finer mesh

### 3.1.1 First create a finer mesh

Use the refinement factor below to increase the mesh resolution. The refinement factor 'refine_fac' multiplies the grid size in each i,j,k direction. You can also adjust the near wall cell size; 'ywall' sets the wall normal cell height for the cell adjacent to the viscous walls.

In [9]:
import matplotlib.pyplot as plt
import timeit

from meshing.read_profile   import *
from meshing.blade_mesher import *
from meshing.read_case import *

# pop-out figure
%matplotlib qt

# mesh refinement inputs
refine_fac = 4.0 # the grid dimensions are scaled by this number in both directions
ywall = 0.01/150 # the final near wall cell height
msmooths = 100 # number of iterations for the poisson solver

# set case names
old_casename = 'notebook2'
new_casename = 'notebook3'

# read old casefiles
case = read_case(old_casename)

# case name and profile coordinates
casename = new_casename
case['casename'] = casename
mesh   = {}
mesh['case'] = casename
mesh['profile'] = 'geom/profile.txt'

# initial mesh inputs
mesh['npp'] = 2*[None]
mesh['ywall'] = 2*[None]
mesh['msmooths'] = 2*[None]

mesh['npp'][0] = 4
mesh['Lup'] = 1.0
mesh['Ldn'] = 1.0
mesh['Lo'] = 0.2
mesh['ywall'][0] = 0.001
mesh['msmooths'][0] = 500
mesh['cor_fac'] = 0.75 # factor shrinking multi-block corner cells - lower values give more shrinkage.

# final mesh inputs
mesh['npp'][1] = 28 # sets minimum points in any direction in smallest block
mesh['refine_fac'] = refine_fac 
mesh['ywall'][1] =  ywall
mesh['msmooths'][1] = msmooths

# scale factor if used (set to 1.0 if not)
mesh['scale_factor'] = 1.0

# flip y coordinates
mesh['flip'] = 1

# modify solver inputs 
solver = case['solver']
solver['niter'] = 100 # run for a few iterations to check all ok
solver['irestart'] = 1 # 1 for restart
solver['istats'] = 0 # switch-off statistics
solver['span'] = 0.1 # spanwise extent for 3D calcs - typically 10-20% of chord
solver['nk'] = 25 # new spanwise points for 3D calculation 
bcs = case['bcs']
gas = case['gas']


# create mesh and initial case files
start = timeit.default_timer()
blk = blade_mesher(mesh,bcs,gas,solver)
stop = timeit.default_timer()

print('Time: ', stop - start)  




gpu input file found
2
Writing inputs for 3dns_gpu
Total ij points
969808
Total GPUs
1
Time:  206.96194834000198


### 3.1.2 Interpolate onto the fine mesh

Now we can interpolate the solution from the coarse mesh onto the new fine mesh. 


In [12]:
from postpro.interp_flo import *
from meshing.write_flo import *
from meshing.write_case import *

# interpolate old flow files onto new grid
prop = interp_flo(old_casename,new_casename)

# write the new flo files
write_flo(prop,new_casename)





gpu input file found
2
gpu input file found
2


Check the case

In [24]:
from postpro.read_flo import *
from postpro.plot_flo import *

# case name
casename = 'notebook3'

# read flo
prop,geom=read_flo(casename)




gpu input file found
2


In [25]:

plot_flo(prop,geom,'vortz',[-1000,1000])
#plot_flo(prop,geom,'p',[0.96e5,1.0e5])
#plot_flo(prop,geom,'po',[0.96e5,1.0e5])


<module 'matplotlib.pyplot' from '/home/aw329/.local/lib/python3.6/site-packages/matplotlib/pyplot.py'>

In [26]:
wall_blocks = [3,4,5,7]

plt.figure(1)
for i in wall_blocks:
    plt.plot(geom[i-1]['x'][:,-1],prop[i-1]['p'][:,-1,0],'-k.')

plt.show()


In [23]:
## Plot the monitor file

from postpro.plot_monitor import *

plot_monitor(casename);

gpu input file found
2


In [29]:
## Check spectrum

from postpro.plot_spectrum import *

plot_spectrum(casename,[1,2],20);

gpu input file found
2
